In [1]:
import pandas as pd


In [2]:
df_planning =pd.read_excel('omloopplanning.xlsx')
df_regeling = pd.read_excel('Connexxion data - 2024-2025.xlsx')
df_afstanden = pd.read_excel('Connexxion data - 2024-2025 - kopie.xlsx')


In [3]:
df_planning.head(1)

,Unnamed: 0,startlocatie,eindlocatie,starttijd,eindtijd,activiteit,buslijn,energieverbruik,starttijd datum,eindtijd datum,omloop nummer
0,0,ehvgar,ehvbst,05:03:00,05:07:00,materiaal rit,NaN,1.98,2024-08-29 05:03:00,2024-08-29 05:07:00,1


In [4]:
df_planning.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

In [5]:
df_planning.head(1)

,index,startlocatie,eindlocatie,starttijd,eindtijd,activiteit,buslijn,energieverbruik,starttijd datum,eindtijd datum,omloop nummer
0,0,ehvgar,ehvbst,05:03:00,05:07:00,materiaal rit,NaN,1.98,2024-08-29 05:03:00,2024-08-29 05:07:00,1


In [3]:

Vertrektijd_locatie = df_regeling[['vertrektijd','startlocatie']]
Vertrektijd_locatie

,vertrektijd,startlocatie
0,06:04,ehvapt
1,06:34,ehvapt
2,06:56,ehvapt
3,07:11,ehvapt
4,07:26,ehvapt
...,...,...
322,18:52,ehvbst
323,18:54,ehvbst
324,19:07,ehvbst
325,19:22,ehvbst


In [4]:
omlopen = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
buslijn = df_planning['buslijn'].unique()
locaties = df_planning['startlocatie'].unique()
activiteit = df_planning['activiteit'].unique()
omloop = df_planning['omloop nummer'].unique()

In [5]:
# Gegevens van de routes
routes = [
    {
        "startlocatie": "ehvapt",
        "eindlocatie": "ehvbst",
        "min_reistijd": 21,
        "max_reistijd": 23,
        "afstand_meters": 10250,
        "buslijn": 400
    },
    {
        "startlocatie": "ehvbst",
        "eindlocatie": "ehvapt",
        "min_reistijd": 21,
        "max_reistijd": 23,
        "afstand_meters": 10708,
        "buslijn": 400
    },
    {
        "startlocatie": "ehvapt",
        "eindlocatie": "ehvbst",
        "min_reistijd": 22,
        "max_reistijd": 25,
        "afstand_meters": 9050,
        "buslijn": 401
    },
    {
        "startlocatie": "ehvbst",
        "eindlocatie": "ehvapt",
        "min_reistijd": 22,
        "max_reistijd": 24,
        "afstand_meters": 9003,
        "buslijn": 401
    },
    {
        "startlocatie": "ehvapt",
        "eindlocatie": "ehvbst",
        "min_reistijd": 20,
        "max_reistijd": 20,
        "afstand_meters": 8600,
        "buslijn": None
    },
    {
        "startlocatie": "ehvbst",
        "eindlocatie": "ehvapt",
        "min_reistijd": 20,
        "max_reistijd": 20,
        "afstand_meters": 8600,
        "buslijn": None
    },
    {
        "startlocatie": "ehvbst",
        "eindlocatie": "ehvgar",
        "min_reistijd": 4,
        "max_reistijd": 4,
        "afstand_meters": 1650,
        "buslijn": None
    },
    {
        "startlocatie": "ehvgar",
        "eindlocatie": "ehvbst",
        "min_reistijd": 4,
        "max_reistijd": 4,
        "afstand_meters": 1650,
        "buslijn": None
    },
    {
        "startlocatie": "ehvapt",
        "eindlocatie": "ehvgar",
        "min_reistijd": 20,
        "max_reistijd": 20,
        "afstand_meters": 9000,
        "buslijn": None
    },
    {
        "startlocatie": "ehvgar",
        "eindlocatie": "ehvapt",
        "min_reistijd": 20,
        "max_reistijd": 20,
        "afstand_meters": 9000,
        "buslijn": None
    },
]

def geef_route_informatie(startlocatie, eindlocatie):
    """Geeft de route-informatie tussen twee locaties."""
    for route in routes:
        if route['startlocatie'] == startlocatie and route['eindlocatie'] == eindlocatie:
            return route
    return f"Geen route gevonden van {startlocatie} naar {eindlocatie}."

# Voorbeeld gebruik
informatie = geef_route_informatie('ehvapt', 'ehvbst')
print(f"Route-informatie van ehvapt naar ehvbst: {informatie}")


Route-informatie van ehvapt naar ehvbst: {'startlocatie': 'ehvapt', 'eindlocatie': 'ehvbst', 'min_reistijd': 21, 'max_reistijd': 23, 'afstand_meters': 10250, 'buslijn': 400}


In [6]:
def zoek_afstand(startlocatie, eindlocatie, buslijn, df_afstanden):
    """Zoekt de afstand tussen start- en eindlocatie voor een specifieke buslijn op in df_afstanden."""
    match = df_afstanden[
        (df_afstanden['startlocatie'] == startlocatie) &
        (df_afstanden['eindlocatie'] == eindlocatie) &
        (df_afstanden['buslijn'] == buslijn)
    ]
    if not match.empty:
        return match.iloc[0]['afstand in meters']
    else:
        return None  # Geen afstand gevonden

# Functie om het energieverbruik te berekenen
def bereken_energieverbruik(buslijn, df_planning, df_afstanden):
    """
    Berekent het energieverbruik voor een opgegeven buslijn door afstanden op te zoeken in df_afstanden
    en deze te vermenigvuldigen met 2.5 kWh per kilometer.
    """
    # Filter de data voor de specifieke buslijn en relevante activiteiten
    df_filtered = df_planning[
        (df_planning['buslijn'] == buslijn) & 
        (df_planning['activiteit'].isin(['dienst rit', 'materiaal rit']))
    ]

    # Constante omzetten naar kWh per meter
    kwh_per_meter = 2.5 / 1000

    totaal_afstand_meters = 0

    # Loop door de gefilterde rijen in df_planning en zoek de afstand op in df_afstanden
    for _, row in df_filtered.iterrows():
        startlocatie = row['startlocatie']
        eindlocatie = row['eindlocatie']
        
        # Zoek de afstand in df_afstanden
        afstand = zoek_afstand(startlocatie, eindlocatie, buslijn, df_afstanden)
        
        if afstand is not None:
            totaal_afstand_meters += afstand
        else:
            print(f"Geen afstand gevonden voor route van {startlocatie} naar {eindlocatie} met buslijn {buslijn}.")

    # Bereken het energieverbruik
    energieverbruik = totaal_afstand_meters * kwh_per_meter
    return energieverbruik

# Voorbeeld gebruik
energieverbruik_400 = bereken_energieverbruik(400, df_planning, df_afstanden)
print(f"Energieverbruik van buslijn 400: {energieverbruik_400:.2f} kWh")

energieverbruik_401 = bereken_energieverbruik(401, df_planning, df_afstanden)
print(f"Energieverbruik van buslijn 401: {energieverbruik_401:.2f} kWh")

Energieverbruik van buslijn 400: 3955.25 kWh
Energieverbruik van buslijn 401: 3971.54 kWh


In [7]:
# ----------------------------- Stap 1: Definieer de parameters -----------------------------

# Parameterset
parameters = {
    "max_reistijd": 30,  # Maximale reistijd in minuten
    "max_afstand": 15000,  # Maximale afstand in meters
    "min_reistijd": 5,  # Minimale reistijd in minuten
    "buslijnen": [400, 401],  # Geïnteresseerde buslijnen
    "startlocatie": "ehvapt",  # Startlocatie voor het filteren
    "eindlocatie": "ehvbst"  # Eindlocatie voor het filteren
}

# ----------------------------- Stap 3: Conversie van tijden -----------------------------

# Conversie van vertrektijden naar datetime objecten
df_regeling['vertrektijd'] = pd.to_datetime(df_regeling['vertrektijd'], format='%H:%M').dt.time

# ----------------------------- Stap 4: Merge en berekeningen -----------------------------

# Merge de DataFrames op start- en eindlocaties
merged_df = pd.merge(df_regeling, df_afstanden, on=['startlocatie', 'eindlocatie'], how='left')

# Bereken de reistijden in minuten
merged_df['reistijd in min'] = (merged_df['max reistijd in min'] + merged_df['min reistijd in min']) / 2

# ----------------------------- Stap 5: Filteren op parameters -----------------------------

# Filteren op buslijnen en reistijd/distance voorwaarden
filtered_df = merged_df[
    (merged_df['buslijn_x'].isin(parameters["buslijnen"])) &
    (merged_df['reistijd in min'] <= parameters["max_reistijd"]) &
    (merged_df['afstand in meters'] <= parameters["max_afstand"]) &
    (merged_df['reistijd in min'] >= parameters["min_reistijd"])
]

# ----------------------------- Stap 6: Resultaten weergeven -----------------------------

# Toont de resultaten
result_columns = ['startlocatie', 'eindlocatie', 'vertrektijd', 'reistijd in min', 'afstand in meters', 'buslijn_x']
print(filtered_df[result_columns].reset_index(drop=True))

    startlocatie eindlocatie vertrektijd  reistijd in min  afstand in meters  \
0         ehvapt      ehvbst    06:04:00             22.0              10250   
1         ehvapt      ehvbst    06:04:00             23.5               9050   
2         ehvapt      ehvbst    06:04:00             20.0               8600   
3         ehvapt      ehvbst    06:34:00             22.0              10250   
4         ehvapt      ehvbst    06:34:00             23.5               9050   
..           ...         ...         ...              ...                ...   
976       ehvbst      ehvapt    19:22:00             23.0               9003   
977       ehvbst      ehvapt    19:22:00             20.0               8600   
978       ehvbst      ehvapt    19:37:00             22.0              10708   
979       ehvbst      ehvapt    19:37:00             23.0               9003   
980       ehvbst      ehvapt    19:37:00             20.0               8600   

     buslijn_x  
0          401  
1    

In [8]:
import numpy as np
import random

In [9]:
import pandas as pd
import numpy as np
import random

# Functie om de totale reistijd te berekenen
def calculate_fitness(individual):
    total_time = 0
    for i in range(len(individual) - 1):
        start = df_regeling.iloc[individual[i]]
        end = df_regeling.iloc[individual[i + 1]]
        
        # Vind de reistijd
        route = df_afstanden[(df_afstanden['startlocatie'] == start['startlocatie']) & 
                              (df_afstanden['eindlocatie'] == end['eindlocatie']) & 
                              (df_afstanden['buslijn'] == start['buslijn'])]
        if route.empty:
            return float('inf')  # Als er geen route is, geef een hoge kost

        total_time += route['min reistijd in min'].values[0]  # Gebruik min reistijd
    return total_time

# Simulated Annealing
def simulated_annealing(max_iterations):
    # Start met een willekeurige oplossing
    current_solution = list(range(len(df_regeling)))
    current_cost = calculate_fitness(current_solution)
    
    # Start met een hoge temperatuur
    temperature = 1000
    cooling_rate = 0.995  # Hoe snel de temperatuur afneemt
    stop_temperature = 1  # Wanneer te stoppen

    iteration = 0  # Initialiseer de iteratieteller

    while temperature > stop_temperature and iteration < max_iterations:
        # Genereer een buuroplossing door twee busritten te verwisselen
        new_solution = current_solution[:]
        i, j = random.sample(range(len(new_solution)), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        new_cost = calculate_fitness(new_solution)
        
        # Bereken de kostenverschil
        cost_difference = new_cost - current_cost
        
        # Als de nieuwe oplossing beter is of met een bepaalde waarschijnlijkheid geaccepteerd wordt
        if cost_difference < 0 or random.uniform(0, 1) < np.exp(-cost_difference / temperature):
            current_solution = new_solution
            current_cost = new_cost
        
        # Verlaag de temperatuur
        temperature *= cooling_rate
        
        # Verhoog de iteratieteller
        iteration += 1
    
    return current_solution, current_cost

# Definieer het maximum aantal iteraties
max_iterations = 10  # Bijvoorbeeld 1000 iteraties

# Run Simulated Annealing
best_schedule, best_time = simulated_annealing(max_iterations)

# Resultaten
print("Beste volgorde van busritten:", best_schedule)
print("Totale reistijd (in minuten):", best_time)


Beste volgorde van busritten: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [10]:
from datetime import datetime, timedelta

In [11]:
import pandas as pd
import random
from datetime import datetime, timedelta

def generate_random_schedule(num_entries):
    # Dummy function om willekeurige data te genereren
    data = {
        'startlocatie': [],
        'eindlocatie': [],
        'starttijd': [],
        'eindtijd': [],
        'activiteit': [],
        'buslijn': [],
        'energieverbruik': [],
        'starttijd datum': [],
        'eindtijd datum': [],
        'omloop nummer': []
    }

    for i in range(num_entries):
        startlocatie = random.choice(['ehvgar', 'ehvbst', 'ehvapt'])
        eindlocatie = random.choice(['ehvgar', 'ehvbst', 'ehvapt'])
        starttijd = datetime.strptime(f'05:{random.randint(0, 59):02d}:{random.randint(0, 59):02d}', '%H:%M:%S').time()
        eindtijd = (datetime.combine(datetime.today(), starttijd) + timedelta(minutes=random.randint(5, 30))).time()
        activiteit = random.choice(['materiaal rit', 'dienst rit', 'idle'])
        buslijn = random.choice(['401', '402', '403', ''])
        energieverbruik = round(random.uniform(0.01, 10.86), 4)

        # Voeg de gegevens toe aan de lijsten
        data['startlocatie'].append(startlocatie)
        data['eindlocatie'].append(eindlocatie)
        data['starttijd'].append(starttijd)
        data['eindtijd'].append(eindtijd)
        data['activiteit'].append(activiteit)
        data['buslijn'].append(buslijn)
        data['energieverbruik'].append(energieverbruik)
        starttijd_datum = datetime.now().replace(hour=starttijd.hour, minute=starttijd.minute, second=starttijd.second, microsecond=0)
        eindtijd_datum = datetime.now().replace(hour=eindtijd.hour, minute=eindtijd.minute, second=eindtijd.second, microsecond=0)
        data['starttijd datum'].append(starttijd_datum)
        data['eindtijd datum'].append(eindtijd_datum)
        data['omloop nummer'].append(1)  # Placeholder voor omloopnummer

    return data

def create_schedule_dataframe(num_entries):
    # Genereer de willekeurige data
    schedule_data = generate_random_schedule(num_entries)
    
    # Creëer de DataFrame
    df = pd.DataFrame(schedule_data)
    
    # Zet de tijdstempels om naar datetime-objecten
    df['starttijd'] = pd.to_datetime(df['starttijd'].astype(str), format='%H:%M:%S').dt.time
    df['eindtijd'] = pd.to_datetime(df['eindtijd'].astype(str), format='%H:%M:%S').dt.time
    
    # Print de DataFrame
    print(df)

    # Opslaan naar een CSV-bestand
    df.to_csv('output_meta_heuristiek.csv', index=False)

# Voer de functie uit om een schema te genereren
create_schedule_dataframe(10)  # Genereer 10 entries


  startlocatie eindlocatie starttijd  eindtijd     activiteit buslijn  \
0       ehvgar      ehvbst  05:42:00  05:57:00  materiaal rit     403   
1       ehvapt      ehvgar  05:49:06  06:03:06     dienst rit     403   
2       ehvgar      ehvapt  05:45:16  06:02:16           idle     401   
3       ehvgar      ehvapt  05:49:06  06:07:06           idle     403   
4       ehvgar      ehvgar  05:19:41  05:35:41  materiaal rit     401   
5       ehvbst      ehvgar  05:37:07  05:45:07     dienst rit     402   
6       ehvgar      ehvgar  05:04:19  05:11:19           idle     402   
7       ehvbst      ehvapt  05:38:55  05:58:55  materiaal rit     401   
8       ehvbst      ehvbst  05:51:24  06:02:24     dienst rit     403   
9       ehvbst      ehvgar  05:44:27  06:13:27           idle     402   

   energieverbruik     starttijd datum      eindtijd datum  omloop nummer  
0           8.3179 2024-10-31 05:42:00 2024-10-31 05:57:00              1  
1           8.7822 2024-10-31 05:49:06 2024-

In [12]:
import pandas as pd
import random
from datetime import datetime, timedelta

def is_within_energy_limit(total_energy, energy_limit):
    return total_energy + energy_limit <= energy_limit

def generate_random_schedule(num_entries, max_energy_per_entry, max_total_energy):
    data = {
        'startlocatie': [],
        'eindlocatie': [],
        'starttijd': [],
        'eindtijd': [],
        'activiteit': [],
        'buslijn': [],
        'energieverbruik': [],
        'starttijd datum': [],
        'eindtijd datum': [],
        'omloop nummer': []
    }
    
    total_energy_consumed = 0  # Houd het totale energieverbruik bij

    for i in range(num_entries):
        # Genereer een willekeurig energieverbruik
        energieverbruik = round(random.uniform(0.01, max_energy_per_entry), 4)

        # Controleer of het energieverbruik binnen het maximum blijft
        if total_energy_consumed + energieverbruik > max_total_energy:
            continue  # Sla deze iteratie over als we het energieplafond overschrijden

        startlocatie = random.choice(['ehvgar', 'ehvbst', 'ehvapt'])
        eindlocatie = random.choice(['ehvgar', 'ehvbst', 'ehvapt'])
        starttijd = datetime.strptime(f'05:{random.randint(0, 59):02d}:{random.randint(0, 59):02d}', '%H:%M:%S').time()
        eindtijd = (datetime.combine(datetime.today(), starttijd) + timedelta(minutes=random.randint(5, 30))).time()
        activiteit = random.choice(['materiaal rit', 'dienst rit', 'idle'])
        buslijn = random.choice(['401', '402', '403', ''])
        
        # Voeg de gegevens toe aan de lijsten
        data['startlocatie'].append(startlocatie)
        data['eindlocatie'].append(eindlocatie)
        data['starttijd'].append(starttijd)
        data['eindtijd'].append(eindtijd)
        data['activiteit'].append(activiteit)
        data['buslijn'].append(buslijn)
        data['energieverbruik'].append(energieverbruik)
        starttijd_datum = datetime.now().replace(hour=starttijd.hour, minute=starttijd.minute, second=starttijd.second, microsecond=0)
        eindtijd_datum = datetime.now().replace(hour=eindtijd.hour, minute=eindtijd.minute, second=eindtijd.second, microsecond=0)
        data['starttijd datum'].append(starttijd_datum)
        data['eindtijd datum'].append(eindtijd_datum)
        data['omloop nummer'].append(1)  # Placeholder voor omloopnummer
        
        total_energy_consumed += energieverbruik  # Update het totale energieverbruik

    return data

def create_schedule_dataframe(num_entries, max_energy_per_entry, max_total_energy):
    schedule_data = generate_random_schedule(num_entries, max_energy_per_entry, max_total_energy)
    
    # Creëer de DataFrame
    df = pd.DataFrame(schedule_data)
    
    # Zet de tijdstempels om naar datetime-objecten
    df['starttijd'] = pd.to_datetime(df['starttijd'].astype(str), format='%H:%M:%S').dt.time
    df['eindtijd'] = pd.to_datetime(df['eindtijd'].astype(str), format='%H:%M:%S').dt.time
    
    # Print de DataFrame
    print(df)

    # Opslaan naar een CSV-bestand
    df.to_csv('output_meta_heuristiek.csv', index=False)

# Voer de functie uit om een schema te genereren
create_schedule_dataframe(num_entries=10, max_energy_per_entry=10.86, max_total_energy=50)  # Genereer 10 entries


  startlocatie eindlocatie starttijd  eindtijd     activiteit buslijn  \
0       ehvgar      ehvgar  05:18:03  05:46:03           idle           
1       ehvgar      ehvapt  05:58:32  06:13:32  materiaal rit     403   
2       ehvapt      ehvapt  05:07:15  05:33:15  materiaal rit     403   
3       ehvgar      ehvbst  05:12:38  05:20:38  materiaal rit           
4       ehvbst      ehvapt  05:22:38  05:46:38  materiaal rit     403   
5       ehvgar      ehvbst  05:53:47  06:06:47           idle     401   
6       ehvbst      ehvapt  05:19:29  05:39:29           idle     403   
7       ehvbst      ehvgar  05:18:32  05:26:32     dienst rit           

   energieverbruik     starttijd datum      eindtijd datum  omloop nummer  
0           6.2232 2024-10-31 05:18:03 2024-10-31 05:46:03              1  
1           4.4806 2024-10-31 05:58:32 2024-10-31 06:13:32              1  
2           0.0974 2024-10-31 05:07:15 2024-10-31 05:33:15              1  
3           8.0737 2024-10-31 05:12:38

In [13]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Functie om te controleren of het totale energieverbruik binnen de limiet blijft
def is_within_energy_limit(total_energy, energy_limit):
    return total_energy <= energy_limit

# Functie om een willekeurig schema te genereren
def generate_schedule(num_entries, energy_per_km, total_energy_limit, start_times):
    data = {
        'startlocatie': [],
        'eindlocatie': [],
        'starttijd': [],
        'eindtijd': [],
        'activiteit': [],
        'buslijn': [],
        'energieverbruik': [],
        'omloop nummer': [],
        'start_soc': [],
        'eind_soc': [],
        'opladen': []
    }
    
    total_energy_consumed = 0  # Houd het totale energieverbruik bij
    scheduled_rides = []  # Lijst voor ingeplande ritten
    buses_in_charge = []  # Lijst van bussen in opladen

    for starttijd in start_times:
        # Genereer een willekeurige afstand (bijv. tussen 1 en 30 km)
        distance_km = random.randint(1, 30)
        
        # Bereken het energieverbruik op basis van de afstand
        energieverbruik = round(distance_km * energy_per_km, 4)

        # Controleer of het energieverbruik binnen het maximum blijft
        if total_energy_consumed + energieverbruik > total_energy_limit:
            print(f"Maximale energie overschreden voor starttijd {starttijd} met afstand {distance_km} km.")
            continue  # Sla deze iteratie over als we het energieplafond overschrijden

        startlocatie = random.choice(['ehvgar', 'ehvbst', 'ehvapt'])
        eindlocatie = random.choice(['ehvgar', 'ehvbst', 'ehvapt'])
        
        # Verleng de tijdsduur van de rit
        eindtijd = (datetime.combine(datetime.today(), starttijd) + timedelta(minutes=random.randint(20, 60))).time()
        
        # Bepaal de buslijn
        buslijn = random.choice(['400', '401'])  # Beperk buslijnen tot 400 en 401

        # Bepaal de start SOC van de bus
        start_soc = round(random.uniform(0.10 * 270, 1.0 * 270), 2)  # Bus heeft minimaal 10% SOC
        eind_soc = start_soc - energieverbruik
        
        # Controleer of de bus bij het eindstation moet opladen
        opladen = eindlocatie == 'ehvgar' and eind_soc < (0.10 * 270)  # Opladen nodig bij ehvgar

        if opladen:
            # Als opladen nodig is, voeg deze bus toe aan de oplaadlijst
            buses_in_charge.append({'starttijd': starttijd, 'buslijn': buslijn})
            eindtijd = (datetime.combine(datetime.today(), eindtijd) + timedelta(hours=(270 - start_soc) / 450)).time()  # Bereken de tijd om op te laden
        
        # Voeg de rit toe aan scheduled_rides
        scheduled_rides.append({
            'starttijd': starttijd,
            'eindtijd': eindtijd,
            'energieverbruik': energieverbruik,
            'startlocatie': startlocatie,
            'eindlocatie': eindlocatie,
            'activiteit': 'dienst rit',  # Altijd dienst rit voor nu
            'buslijn': buslijn,
            'start_soc': start_soc,
            'eind_soc': eind_soc,
            'opladen': opladen
        })
        
        total_energy_consumed += energieverbruik  # Update het totale energieverbruik

    # Plan de ritten met respect voor de omlopen
    scheduled_data = plan_with_loops(scheduled_rides)

    return scheduled_data

# Functie om ritten in te plannen met respect voor omlopen
def plan_with_loops(scheduled_rides):
    loop_count = 0
    bus_loops = []  # Lijst voor actieve bussen
    output_data = {
        'startlocatie': [],
        'eindlocatie': [],
        'starttijd': [],
        'eindtijd': [],
        'activiteit': [],
        'buslijn': [],
        'energieverbruik': [],
        'omloop nummer': [],
        'start_soc': [],
        'eind_soc': [],
        'opladen': []
    }

    for ride in scheduled_rides:
        assigned = False
        for i, bus in enumerate(bus_loops):
            # Controleer of deze bus beschikbaar is
            if bus['eindtijd'] <= ride['starttijd']:
                # Wijs de rit toe aan deze bus
                bus_loops[i]['eindtijd'] = ride['eindtijd']  # Update de eindtijd van de bus
                output_data['startlocatie'].append(ride['startlocatie'])
                output_data['eindlocatie'].append(ride['eindlocatie'])
                output_data['starttijd'].append(ride['starttijd'])
                output_data['eindtijd'].append(ride['eindtijd'])
                output_data['activiteit'].append(ride['activiteit'])
                output_data['buslijn'].append(ride['buslijn'])
                output_data['energieverbruik'].append(ride['energieverbruik'])
                output_data['omloop nummer'].append(i + 1)  # Omloop nummer
                output_data['start_soc'].append(ride['start_soc'])  # Start SOC
                output_data['eind_soc'].append(ride['eind_soc'])  # Eind SOC
                output_data['opladen'].append(ride['opladen'])  # Opladen status
                assigned = True
                break

        if not assigned:
            # Geen beschikbare bus, maak een nieuwe omloop
            loop_count += 1
            bus_loops.append({'eindtijd': ride['eindtijd']})  # Voeg een nieuwe bus toe
            output_data['startlocatie'].append(ride['startlocatie'])
            output_data['eindlocatie'].append(ride['eindlocatie'])
            output_data['starttijd'].append(ride['starttijd'])
            output_data['eindtijd'].append(ride['eindtijd'])
            output_data['activiteit'].append(ride['activiteit'])
            output_data['buslijn'].append(ride['buslijn'])
            output_data['energieverbruik'].append(ride['energieverbruik'])
            output_data['omloop nummer'].append(loop_count)  # Nieuwe omloop nummer
            output_data['start_soc'].append(ride['start_soc'])  # Start SOC
            output_data['eind_soc'].append(ride['eind_soc'])  # Eind SOC
            output_data['opladen'].append(ride['opladen'])  # Opladen status

    return output_data

# Functie om het schema DataFrame te creëren
def create_schedule_dataframe(num_entries, energy_per_km, total_energy_limit, df_regeling):
    start_times = df_regeling['starttijd'].tolist()  # Lijst met starttijden (al in tijdformaat)
    schedule_data = generate_schedule(num_entries, energy_per_km, total_energy_limit, start_times)
    
    # Creëer de DataFrame
    df = pd.DataFrame(schedule_data)
    
    # Zet de tijdstempels om naar datetime-objecten met de huidige datum
    df['starttijd'] = df['starttijd'].apply(lambda x: datetime.combine(datetime.today(), x))
    df['eindtijd'] = df['eindtijd'].apply(lambda x: datetime.combine(datetime.today(), x))

    # Print de DataFrame
    print(df)

    # Opslaan naar een Excel-bestand
    df.to_excel('output_meta_heuristiek.xlsx', index=False)

# Voorbeeld DataFrame voor df_regeling met starttijden
df_regeling = pd.DataFrame({
    'starttijd': ['05:00:00', '05:30:00', '06:00:00', '06:30:00', '07:00:00'] * 145  # 721 rijen
})
df_regeling['starttijd'] = pd.to_datetime(df_regeling['starttijd']).dt.time

# Maximale totale energie die beschikbaar is (270 kWh - 10% = 243 kWh)
total_energy_limit = 270 - (0.10 * 270)

# Voer de functie uit om een schema te genereren
create_schedule_dataframe(num_entries=len(df_regeling), 
                           energy_per_km=2.5, 
                           total_energy_limit=total_energy_limit, 
                           df_regeling=df_regeling)  # Genereer ritten voor de gegeven starttijden


Maximale energie overschreden voor starttijd 05:30:00 met afstand 19 km.
Maximale energie overschreden voor starttijd 06:30:00 met afstand 26 km.
Maximale energie overschreden voor starttijd 07:00:00 met afstand 4 km.
Maximale energie overschreden voor starttijd 05:00:00 met afstand 29 km.
Maximale energie overschreden voor starttijd 05:30:00 met afstand 8 km.
Maximale energie overschreden voor starttijd 06:00:00 met afstand 19 km.
Maximale energie overschreden voor starttijd 06:30:00 met afstand 19 km.
Maximale energie overschreden voor starttijd 07:00:00 met afstand 6 km.
Maximale energie overschreden voor starttijd 05:00:00 met afstand 16 km.
Maximale energie overschreden voor starttijd 05:30:00 met afstand 23 km.
Maximale energie overschreden voor starttijd 06:00:00 met afstand 26 km.
Maximale energie overschreden voor starttijd 06:30:00 met afstand 28 km.
Maximale energie overschreden voor starttijd 07:00:00 met afstand 20 km.
Maximale energie overschreden voor starttijd 05:00:00 

C:\Users\mikha\AppData\Local\Temp\ipykernel_2604\3815103372.py:162: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_regeling['starttijd'] = pd.to_datetime(df_regeling['starttijd']).dt.time


In [14]:
import pandas as pd
import numpy as np

# Aannames
max_battery_capacity = 270  # kWh, de maximale batterijcapaciteit van de bus
min_soc_required = 0.10 * max_battery_capacity  # 10% SOC, minimaal benodigde energie bij aankomst
charging_rate = 450  # kWh per uur, oplaadsnelheid
charging_slots = 20  # Aantal bussen dat tegelijk kan opladen
energy_per_km = 2.5  # Energieverbruik per kilometer, pas deze waarde aan indien nodig

# Laad de DataFrame df_regeling (voeg je DataFrame hier in)
# df_regeling = ...

def is_schedule_valid(distance_meters, current_energy):
    # Bereken de benodigde energie voor de rit
    distance_km = distance_meters / 1000  # Omzetten naar kilometers
    required_energy = distance_km * energy_per_km
    
    # Controleer of de bus voldoende energie heeft
    if current_energy - required_energy < min_soc_required:
        return False
    return True

def create_schedule_dataframe(df_regeling):
    # Huidige energie bij vertrek (bijvoorbeeld beginnen met een volle batterij)
    current_energy = max_battery_capacity
    
    schedule = []
    
    for index, row in df_regeling.iterrows():
        start_time = row['starttijd']
        distance_meters = row['afstand_meters']  # Afstand in meters
        
        # Controleer of de rit toegestaan is
        if is_schedule_valid(distance_meters, current_energy):
            distance_km = distance_meters / 1000  # Omzetten naar kilometers
            travel_time = row['min_reistijd']  # Aangenomen dat dit in minuten is
            end_time = start_time + pd.Timedelta(minutes=travel_time)
            
            # Voeg de rit toe aan de planning
            schedule.append({
                'starttijd': start_time,
                'eindtijd': end_time,
                'startlocatie': row['startlocatie'],
                'eindlocatie': row['eindlocatie'],
                'buslijn': row['buslijn_x'],
                'energieverbruik': distance_km * energy_per_km,
            })
            
            # Update de huidige energie
            current_energy -= distance_km * energy_per_km
            
            # Controleer of de bus moet opladen na de rit
            if current_energy < min_soc_required:
                # Oplaadtijd berekenen
                charging_time_needed = (max_battery_capacity - current_energy) / charging_rate * 60  # Omrekeningen naar minuten
                current_energy = max_battery_capacity  # Veronderstel dat de bus volledig is opgeladen na het opladen
                
                # Voeg idle tijd toe in de planning
                idle_end_time = end_time + pd.Timedelta(minutes=charging_time_needed)
                schedule.append({
                    'starttijd': end_time,
                    'eindtijd': idle_end_time,
                    'startlocatie': row['eindlocatie'],  # Waar de bus oplaadt
                    'eindlocatie': row['eindlocatie'],
                    'buslijn': 'Opladen',
                    'energieverbruik': 0,  # Geen energieverbruik voor idle
                })

    # Maak een DataFrame van de planning
    df_schedule = pd.DataFrame(schedule)
    return df_schedule

# Voer de functie uit om een schema te genereren
df_schedule = create_schedule_dataframe(df_regeling)

# Exporteer naar Excel
output_filename = 'output_meta_heuristiek_v2.xlsx'
df_schedule.to_excel(output_filename, index=False)

print("Schema succesvol aangemaakt en opgeslagen als", output_filename)


KeyError: 'afstand_meters'